In [34]:
import requests
import time
import csv
from bs4 import BeautifulSoup

In [3]:
train_nos = ['01025',]
# for train_no in train_nos:
train_no = '01025'

In [ ]:
BASE_URL = f"https://www.ixigo.com/trains/{train_no}"
# base_url = "https://www.railyatri.in/time-table/{}"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.4567.89 Safari/537.36'
}
r = requests.get(BASE_URL)
soup = BeautifulSoup(r.text, 'html.parser')

sql1 = ""
values1 = []

source_station = ''
destination_station = ''
        
table = soup.find('table', class_='schedule-table').contents
tbody = table[1].contents
for d in tbody:
    div = d.contents
    tr = div[0].contents
    if tr[2].text == 'starts':
        source_station = tr[1].text
    if tr[3].text == 'ends':
        destination_station = tr[1].text


train_name = soup.find('span', class_='name').text.rstrip(f' {train_no} Train')
row_data = soup.find_all('td', class_='train-detail-table-r-data')
classes = row_data[0].text
days = row_data[1].text.split(',')
DAYS = [' Mon',' Tue',' Wed',' Thu',' Fri',' Sat',' Sun']
Mon = Tue = Wed = Thu = Fri = Sat = Sun = 'F'
for day in days:
    if day == ' Mon':
        Mon = 'T'
    elif day == ' Tue':
        Tue = 'T'
    elif day == ' Wed':
        Wed = 'T'
    elif day == ' Thu':
        Thu = 'T'
    elif day == ' Fri':
        Fri = 'T'
    elif day == ' Sat':
        Sat = 'T'
    elif day == ' Sun':
        Sun = 'T'

In [ ]:
sql2 = "INSERT INTO trains (train_number, train_name, source_station, destination_station, mon, tue, wed, thu, fri, sat, sun, classes) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,);"
values2 = (train_no, train_name, source_station, destination_station, Mon, Tue, Wed, Thu, Fri, Sat, Sun, classes)

# cursor.execute(sql2, values2)
# db.commit()

In [ ]:
# print('Train Details :- \n')
# print('Train No. — ',train_no,'\tTrain Name — ',train_name)
# print('Mon\tTue\tWed\tThu\tFri\tSat\tSun\tClasses')
# for tick in days_tick:
#     print(tick,'\t',end='')
# print(classes)


Train Details :- 

Train No. —  01025 	Train Name —  Dr Ballia Spl
Mon	Tue	Wed	Thu	Fri	Sat	Sun	Classes
✔ 	— 	✔ 	— 	✔ 	— 	— 	2A, 3A, SL


In [ ]:
source_station

'Dadar'

In [ ]:
destination_station

'Ballia'

In [ ]:
# db.close()

In [35]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.ixigo.com/trains/{}"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def scrape_train(train_no):
    url = BASE_URL.format(train_no)
    r = requests.get(url, headers=HEADERS)

    if r.status_code != 200:
        print("Failed to load page")
        return None

    soup = BeautifulSoup(r.text, "html.parser")

    tables = soup.find_all("table")

    # -------- Extract Train Info --------
    if soup.find('span', class_='name') == None:
        time.sleep(10)
        scrape_train(train_no)
    else:
        train_name = soup.find('span', class_='name').text.rstrip(f' {train_no} Train')
    
        info_table = tables[0]
        info_data = {}
    
        for row in info_table.find_all("tr"):
            cols = [c.get_text(strip=True) for c in row.find_all(["td","th"])]
            if len(cols) == 2:
                info_data[cols[0]] = cols[1]
    
        # -------- Extract Schedule --------
        schedule_table = tables[1]
        rows = schedule_table.find_all("tr")[1:]  # skip header
    
        stops = []
    
        for idx, row in enumerate(rows):
            cols = [c.get_text(strip=True) for c in row.find_all("td")]
    
            if len(cols) < 10:
                continue
            
            station_code = cols[0]
            station_name = cols[1]
            arrival_time = None if cols[2].lower() == "starts" else cols[2]
            departure_time = None if cols[3].lower() == "ends" else cols[3]
            day_offset = int(cols[8]) - 1   # ixigo shows Day starting from 1
    
            stops.append({
                "station_code": station_code,
                "station_name": station_name,
                "arrival_time": arrival_time,
                "departure_time": departure_time,
                "day": day_offset
            })
    
        return {
            "train_no": train_no,
            "train_name": train_name,
            "classes": info_data.get("Classes", ""),
            "service_days": info_data.get("Service Days", ""),
            "stops": stops
        }
    

In [ ]:
train_no_list = [
    "11463", "11464", "11465", "11466", "11601", "11602", "11603", "11604",
    "11605", "11606", "11651", "11652", "11701", "11702", "11703", "11704",
    "11705", "11706", "11751", "11752", "11753", "11754", "11755", "11756",
    "11801", "11802", "11807", "11808", "11841", "11842", "11901", "11902",
    "11903", "11904", "11905", "11906", "12001", "12002", "12003", "12004",
    "12005", "12006", "12007", "12008", "12009", "12010", "12011", "12012",
    "12013", "12014", "12015", "12016", "12017", "12018", "12019", "12020",
    "12021", "12022", "12023", "12024", "12025", "12026", "12027", "12028",
    "12029", "12030", "12031", "12032", "12033", "12034", "12035", "12036",
    "12037", "12038", "12039", "12040", "12041", "12042", "12045", "12046",
    "12047", "12048", "12049", "12050", "12051", "12052", "12053", "12054",
    "12055", "12056", "12057", "12058", "12059", "12060", "12061", "12062",
    "12065", "12066", "12067", "12068", "12069", "12070", "12071", "12072",
    "12073", "12074", "12075", "12076", "12077", "12078", "12079", "12080",
    "12081", "12082", "12083", "12084", "12087", "12088", "12089", "12090",
    "12091", "12092", "12097", "12098", "12101", "12102", "12103", "12104",
    "12105", "12106", "12107", "12108", "12109", "12110", "12111", "12112",
    "12113", "12114", "12115", "12116", "12119", "12120", "12121", "12122",
    "12123", "12124", "12125", "12126", "12127", "12128", "12129", "12130",
    "12131", "12132", "12133", "12134", "12135", "12136", "12137", "12138",
    "12139", "12140", "12141", "12142", "12143", "12144", "12145", "12146",
    "12147", "12148", "11401", "11402", "12151", "12152", "12153", "12154",
    "12155", "12156", "12157", "12158", "12159", "12160", "12161", "12162",
    "12163", "12164", "12165", "12166", "12167", "12168", "12169", "12170",
    "12171", "12172", "12173", "12174", "12175", "12176", "12177", "12178",
    "12179", "12180", "12181", "12182", "12183", "12184", "12185", "12186",
    "12187", "12188", "12189", "12190", "12191", "12192", "12193", "12194",
    "12195", "12196", "12197", "12198", "12201", "12202", "12203", "12204",
    "12207", "12208", "12209", "12210", "12211", "12212", "12213", "12214",
    "12215", "12216", "12217", "12218", "12219", "12220", "12221", "12222",
    "12223", "12224", "12225", "12226", "12227", "12228", "12229", "12230",
    "12231", "12232", "12235", "12236", "12237", "12238", "12239", "12240",
    "12243", "12244", "12245", "12246", "12247", "12248", "12249", "12250",
    "12251", "12252", "12253", "12254", "12257", "12258", "12259", "12260",
    "12261", "12262", "12263", "12264", "12265", "12266", "12267", "12268",
    "12269", "12270", "12273", "12274", "12275", "12276", "12277", "12278",
    "12279", "12280", "12281", "12282", "12283", "12284", "12285", "12286",
    "12289", "12290", "12291", "12292", "12293", "12294", "12295", "12296",
    "12297", "12298", "12301", "12302", "12303", "12304", "12305", "12306",
    "12307", "12308", "12309", "12310", "12311", "12312", "12313", "12314",
    "12315", "12316", "12317", "12318", "12319", "12320", "12321", "12322",
    "12323", "12324", "12325", "12326", "12327", "12328", "12329", "12330",
    "12331", "12332", "12333", "12334", "12335", "12336", "12337", "12338",
    "12339", "12340", "12341", "12342", "12343", "12344", "12345", "12346",
    "12347", "12348", "12349", "12350", "12351", "12352", "12353", "12354",
    "12355", "12356", "12357", "12358", "12361", "12362", "12363", "12364",
    "12365", "12366", "12367", "12368", "12369", "12370", "12371", "12372",
    "12375", "12376", "12377", "12378", "12379", "12380", "12381", "12382",
    "12383", "12384", "12389", "12390", "12391", "12392", "12393", "12394",
    "12395", "12396", "12397", "12398", "12401", "12402", "12403", "12404",
    "12405", "12406", "12407", "12408", "12409", "12410", "12411", "12412",
    "12413", "12414", "12415", "12416", "12417", "12418", "12419", "12420",
    "12421", "12422", "12423", "12424", "12425", "12426", "12427", "12428",
    "12429", "12430", "12431", "12432", "12433", "12434", "12435", "12436",
    "12437", "12438", "12439", "12440", "12441", "12442", "12443", "12444",
    "12445", "12446", "12447", "12448", "12449", "12450", "12451", "12452",
    "12453", "12454", "12455", "12456", "12457", "12458", "12461", "12462",
    "12463", "12464", "12465", "12466", "12467", "12468", "12469", "12470",
    "12471", "12472", "12473", "12474", "12475", "12476", "12477", "12478",
    "12479", "12480", "12481", "12482", "12483", "12484", "12485", "12486",
    "12487", "12488", "12489", "12490", "12493", "12494", "12495", "12496",
    "12497", "12498", "12501", "12502", "12503", "12504", "12505", "12506",
    "12507", "12508", "12509", "12510", "12511", "12512", "12513", "12514",
    "12515", "12516", "12517", "12518", "12519", "12520", "12521", "12522",
    "12523", "12524", "12533", "12534", "12535", "12536", "12539", "12540",
    "12545", "12546", "12547", "12548", "12549", "12550", "12551", "12552",
    "12555", "12556", "12557", "12558", "12559", "12560", "12561", "12562",
    "12565", "12566", "12571", "12572", "12577", "12578", "12581", "12582",
    "12583", "12584", "12587", "12588", "12589", "12590", "12591", "12592",
    "12593", "12594", "12595", "12596", "12597", "12598", "12601", "12602",
    "12603", "12604", "12605", "12606", "12607", "12608", "12611", "12612",
    "12613", "12614", "12615", "12616", "12617", "12618", "12619", "12620",
    "12621", "12622", "12623", "12624", "12625", "12626", "12627", "12628",
    "12629", "12630", "12631", "12632", "12633", "12634", "12635", "12636",
    "12637", "12638", "12639", "12640", "12641", "12642", "12643", "12644",
    "12645", "12646", "12647", "12648", "12649", "12650", "12651", "12652",
    "12653", "12654", "12655", "12656", "12657", "12658", "12659", "12660",
    "12661", "12662", "12663", "12664", "12665", "12666", "12667", "12668",
    "12669", "12670", "12671", "12672", "12673", "12674", "12675", "12676",
    "12679", "12680", "12681", "12682", "12683", "12684", "12685", "12686",
    "12689", "12690", "12691", "12692", "12693", "12694", "12695", "12696",
    "12697", "12698", "12701", "12702", "12703", "12704", "12705", "12706",
    "12707", "12708", "12709", "12710", "12711", "12712", "12713", "12714",
    "12715", "12716", "12717", "12718", "12719", "12720", "12721", "12722",
    "12723", "12724", "12725", "12726", "12727", "12728", "12731", "12732",
    "12733", "12734", "12735", "12736", "12737", "12738", "12739", "12740",
    "12741", "12742", "12743", "12744", "12745", "12746", "12747", "12748",
    "12749", "12750", "12751", "12752", "12753", "12754", "12755", "12756",
    "12757", "12758", "12759", "12760", "12761", "12762", "12763", "12764",
    "12765", "12766", "12767", "12768", "12769", "12770", "12771", "12772",
    "12773", "12774", "12775", "12776", "12777", "12778", "12779", "12780",
    "12781", "12782", "12783", "12784", "12785", "12786", "12787", "12788",
    "12789", "12790", "12791", "12792", "12793", "12794", "12795", "12796",
    "12797", "12798", "12801", "12802", "12803", "12804", "12805", "12806",
    "12807", "12808", "12809", "12810", "12811", "12812", "12813", "12814",
    "12815", "12816", "12817", "12818", "12819", "12820", "12821", "12822",
    "12823", "12824", "12825", "12826", "12827", "12828", "20854", "20853",
    "12831", "12832", "12833", "12834", "12835", "12836", "12837", "12838",
    "12839", "12840", "12841", "12842", "12843", "12844", "12845", "12846",
    "12847", "12848", "12849", "12850", "12851", "12852", "12853", "12854",
    "12855", "12856", "12857", "12858", "12859", "12860", "12861", "12862",
    "12863", "12864", "12867", "12868", "12869", "12870", "12871", "12872",
    "12873", "12874", "12875", "12876", "12877", "12878", "12879", "12880",
    "12881", "12882", "12883", "12884", "12885", "12886", "12887", "12888",
    "12889", "12890", "12891", "12892", "12893", "12894", "12895", "12896",
    "20852", "20851", "12901", "12902", "12903", "12904", "12905", "12906",
    "12907", "12908", "12909", "12910", "12911", "12912", "12913", "12914",
    "12915", "12916", "12917", "12918", "12919", "12920", "12921", "12922",
    "12923", "12924", "12925", "12926", "12927", "12928", "12929", "12930",
    "12931", "12932", "12933", "12934", "12935", "12936", "12937", "12938",
    "12939", "12940", "12941", "12942", "12943", "12944", "12945", "12946",
    "12947", "12948", "12949", "12950", "12951", "12952", "12953", "12954",
    "12955", "12956", "12957", "12958", "12959", "12960", "12961", "12962",
    "12963", "12964", "12965", "12966", "12967", "12968", "12969", "12970",
    "12971", "12972", "12973", "12974", "12975", "12976", "12977", "12978",
    "12979", "12980", "12981", "12982", "12983", "12984", "12985", "12986",
    "12987", "12988", "12989", "12990", "12991", "12992", "12993", "12994",
    "12995", "12996", "12997", "12998", "13005", "13006", "13009", "13010",
    "13011", "13012", "13015", "13016", "13017", "13018", "13019", "13020",
    "13021", "13022", "13023", "13024", "13025", "13026", "13027", "13028",
    "13029", "13030", "13031", "13032", "13033", "13034", "13043", "13044",
    "13045", "13046", "13053", "13054", "13063", "13064", "13071", "13072",
    "13103", "13104", "13105", "13106", "13113", "13114", "13115", "13116",
    "13117", "13118", "13121", "13122", "13125", "13126", "13127", "13128",
    "13131", "13132", "13135", "13136", "13137", "13138", "13141", "13142",
    "13145", "13146", "13147", "13148", "13149", "13150", "13151", "13152",
    "13153", "13154", "13155", "13156", "13157", "13158", "13159", "13160",
    "13161", "13162", "13163", "13164", "13165", "13166", "13167", "13168",
    "13169", "13170", "13173", "13174", "13175", "13176", "13177", "13178",
    "13179", "13180", "13181", "13182", "13185", "13186", "13187", "13188",
    "13189", "13190", "13201", "13202", "13205", "13206", "13207", "13208",
    "13209", "13210", "13211", "13212", "13213", "13214", "13225", "13226",
    "13227", "13228", "13229", "13230", "13233", "13234", "13235", "13236",
    "13237", "13238", "13239", "13240", "13241", "13242", "13243", "13244",
    "13245", "13246", "13247", "13248", "13249", "13250", "13251", "13252",
    "13257", "13258", "13281", "13282", "13287", "13288", "13301", "13302",
    "13303", "13304", "13305", "13306", "13307", "13308", "13309", "13310",
    "13319", "13320", "13329", "13330", "13331", "13332", "13333", "13334",
    "13343", "13344", "13345", "13346", "13347", "13348", "13349", "13350",
    "13351", "13352", "13401", "13402", "13403", "13404", "13409", "13410",
    "13415", "13416", "13417", "13418", "13419", "13420", "13421", "13422",
    "13423", "13424", "13425", "13426", "13427", "13428", "13429", "13430",
    "13431", "13432", "13433", "13434", "13435", "13436", "13465", "13466",
    "13503", "13504", "13505", "13506", "13507", "13508", "13509", "13510",
    "13511", "13512", "13513", "13514", "13545", "13546", "13553", "13554",
    "13697", "13698", "14001", "14002", "14003", "14004", "14005", "14006",
    "14007", "14008", "14009", "14010", "14011", "14012", "14013", "14014",
    "14015", "14016", "14017", "14018", "14023", "14024", "14027", "14028",
    "14029", "14030", "14035", "14036", "14037", "14038", "14341", "14342",
    "14047", "14048", "14049", "14050", "14053", "14054", "14085", "14086",
    "14087", "14088", "14089", "14090", "14101", "14102", "14109", "14110",
    "14111", "14112", "14113", "14114", "14115", "14116", "14117", "14118",
    "14119", "14120", "14123", "14124", "14151", "14152", "14163", "14164",
    "14201", "14202", "14205", "14206", "14207", "14208", "14209", "14210",
    "14211", "14212", "14213", "14214", "14215", "14216", "14217", "14218",
    "14223", "14224", "14229", "14230", "14231", "14232", "14233", "14234",
    "14235", "14236", "14241", "14242", "14259", "14260", "14261", "14262",
    "14303", "14304", "14305", "14306", "14307", "14308", "14309", "14310",
    "14311", "14312", "14313", "14314", "14315", "14316", "14317", "14318",
    "14319", "14320", "14321", "14322", "14323", "14324", "14331", "14332",
    "14503", "14504", "14505", "14506", "14507", "14508", "14509", "14510",
    "14521", "14522", "14523", "14524", "14525", "14526", "14527", "14528",
    "14541", "14542", "14545", "14546", "14601", "14602", "14603", "14604",
    "14605", "14606", "14609", "14610", "14611", "14612", "14613", "14614",
    "14615", "14616", "14617", "14618", "14619", "14620", "14621", "14622",
    "14625", "14626", "14627", "14628", "14629", "14630", "14631", "14632",
    "14633", "14634", "14649", "14650", "14653", "14654", "14661", "14662",
    "14673", "14674", "14679", "14680", "14681", "14682", "14691", "14692",
    "14701", "14702", "14703", "14704", "14705", "14706", "14707", "14708",
    "14713", "14714", "14715", "14716", "14717", "14718", "14719", "14720",
    "14721", "14722", "14725", "14726", "14727", "14728", "14729", "14730",
    "14731", "14732", "14733", "14734", "14735", "14736", "14737", "14738",
    "14795", "14796", "14801", "14802", "14803", "14804", "14805", "14806",
    "14807", "14808", "14813", "14814", "14815", "14816", "14821", "14822",
    "14823", "14824", "14853", "14854", "14863", "14864", "14865", "14866",
    "14887", "14888", "14889", "14890", "14891", "14892", "14893", "14894",
    "14895", "14896", "14897", "14898", "15001", "15002", "15003", "15004",
    "15005", "15006", "15007", "15008", "15009", "15010", "15011", "15012",
    "15013", "15014", "15015", "15016", "15017", "15018", "15019", "15020",
    "15021", "15022", "15023", "15024", "15025", "15026", "15027", "15028",
    "15029", "15030", "15031", "15032", "15033", "15034", "15035", "15036",
    "15037", "15038", "15039", "15040", "15043", "15044", "15045", "15046",
    "15047", "15048", "15049", "15050", "15051", "15052", "15053", "15054",
    "15055", "15056", "15057", "15058", "15059", "15060", "15061", "15062",
    "15063", "15064", "15065", "15066", "15067", "15068", "15069", "15070",
    "15071", "15072", "15073", "15074", "15075", "15076", "15077", "15078",
    "15079", "15080", "15081", "15082", "15083", "15084", "15089", "15090",
    "15091", "15092", "15097", "15098", "15103", "15104", "15105", "15106",
    "15107", "15108", "15109", "15110", "15111", "15112", "15113", "15114",
    "15115", "15116", "15117", "15118", "15119", "15120", "15125", "15126",
    "15127", "15128", "15129", "15130", "15131", "15132", "15133", "15134",
    "15135", "15136", "15159", "15160", "15181", "15182", "15201", "15202",
    "15203", "15204", "15205", "15206", "15211", "15212", "15215", "15216",
    "15227", "15228", "15231", "15232", "15233", "15234", "15235", "15236",
    "15267", "15268", "15269", "15270", "15271", "15272", "15273", "15274",
    "15279", "15280", "15283", "15284", "15293", "15294", "15417", "15418",
    "15463", "15464", "15467", "15468", "15483", "15484", "15501", "15502",
    "15503", "15504", "15505", "15506", "15507", "15508", "15509", "15510",
    "15515", "15516", "15527", "15528", "15529", "15530", "15531", "15532",
    "15549", "15550", "15551", "15552", "15553", "15554", "15555", "15556",
    "15557", "15558", "15559", "15560", "15561", "15562", "15565", "15566",
    "15567", "15568", "15589", "15590", "15601", "15602", "15603", "15604",
    "15609", "15610", "15611", "15612", "15613", "15614", "15615", "15616",
    "15617", "15618", "15619", "15620", "15621", "15622", "15623", "15624",
    "15625", "15626", "15629", "15630", "15631", "15632", "15633", "15634",
    "15635", "15636", "15639", "15640", "15641", "15642", "15643", "15644",
    "15647", "15648", "15651", "15652", "15653", "15654", "15655", "15656",
    "15657", "15658", "15661", "15662", "15663", "15664", "15665", "15666",
    "15667", "15668", "15669", "15670", "15701", "15702", "15703", "15704",
    "15705", "15706", "15707", "15708", "15709", "15710", "15711", "15712",
    "15713", "15714", "15715", "15716", "15719", "15720", "15721", "15722",
    "15723", "15724", "15733", "15734", "15743", "15744", "15753", "15754",
    "15767", "15768", "15769", "15770", "15777", "15778", "15813", "15814",
    "15815", "15816", "15817", "15818", "15888", "15887", "15895", "15896",
    "15901", "15902", "15903", "15904", "15907", "15908", "15909", "15910",
    "15911", "15912", "15925", "15926", "15927", "15928", "15929", "15930",
    "15933", "15934", "15945", "15946", "15959", "15960", "15961", "15962",
    "15967", "15968", "16003", "16004", "16021", "16022", "16031", "16032",
    "16053", "16054", "16057", "16058", "16085", "16086", "16087", "16088",
    "16089", "16090", "16093", "16094", "16101", "16102", "16103", "16104",
    "16111", "16112", "16115", "16116", "16127", "16128", "16159", "16160",
    "16175", "16176", "16179", "16180", "16187", "16188", "16203", "16204",
    "16205", "16206", "16207", "16208", "16209", "16210", "16211", "16212",
    "16213", "16214", "16215", "16216", "16217", "16218", "16219", "16220",
    "16221", "16222", "16225", "16226", "16227", "16228", "16231", "16232",
    "16833", "16834", "16235", "16236", "16239", "16240", "16301", "16302",
    "16303", "16304", "16305", "16306", "16307", "16308", "16309", "16310",
    "16311", "16312", "16315", "16316", "16317", "16318", "16319", "16320",
    "16321", "16322", "16323", "16324", "16325", "16326", "16327", "16328",
    "16331", "16332", "16333", "16334", "16335", "16336", "16337", "16338",
    "16339", "16340", "16341", "16342", "16343", "16344", "16345", "16346",
    "16347", "16348", "16349", "16350", "16351", "16352", "16353", "16354",
    "16355", "16356", "16361", "16362", "16366", "16367", "16368", "16377",
    "16378", "16381", "16382", "20693", "20694", "16511", "16512", "16515",
    "16516", "16519", "16520", "16521", "16522", "16525", "16526", "16527",
    "16528", "16529", "16530", "16531", "16532", "16533", "16534", "16535",
    "16536", "16539", "16540", "16541", "16542", "16545", "16546", "16547",
    "16548", "16549", "16550", "16551", "16552", "16565", "16566", "16567",
    "16568", "16569", "16570", "16561", "16562", "16571", "16572", "16573",
    "16574", "16575", "16576", "16577", "16578", "16581", "16582", "16583",
    "16584", "16585", "16586", "16587", "16588", "16589", "16590", "16591",
    "16592", "16593", "16594", "16595", "16596", "16601", "16602", "16603",
    "16604", "16605", "16606", "16607", "16608", "16609", "16610", "16613",
    "16614", "16615", "16616", "16617", "16618", "16621", "16622", "16629",
    "16630", "16649", "16650", "16721", "16722", "16729", "16730", "16731",
    "16732", "16733", "16734", "16751", "16752", "16765", "16766", "16779",
    "16780", "16787", "16788", "16791", "16792", "16811", "16812", "16843",
    "16844", "16845", "16846", "16847", "16848", "16849", "16850", "16853",
    "16854", "16855", "16856", "16857", "16858", "16861", "16862", "16865",
    "16866", "16867", "16868", "16870", "17001", "17002", "17003", "17004",
    "17005", "17006", "17007", "17008", "17009", "17010", "17011", "17012",
    "17013", "17014", "17015", "17016", "17019", "17020", "17021", "17022",
    "17023", "17024", "17027", "17028", "17029", "17030", "17033", "17034",
    "17035", "17036", "17039", "17040", "17057", "17058", "17063", "17064",
    "17201", "17202", "17205", "17206", "17207", "17208", "17209", "17210",
    "17211", "17212", "17215", "17216", "17219", "17220", "17221", "17222",
    "17225", "17226", "17227", "17228", "17231", "17232", "17233", "17234",
    "17235", "17236", "17237", "17238", "17239", "17240", "17241", "17242",
    "17243", "17244", "17245", "17246", "17247", "17248", "17249", "17250",
    "17251", "17252", "17253", "17254", "17255", "17256", "17257", "17258",
    "17259", "17260", "17261", "17262", "17263", "17264", "17267", "17268",
    "17269", "17270", "17281", "17282", "17301", "17302", "17303", "17304",
    "17307", "17308", "17309", "17310", "17315", "17316", "17317", "17318",
    "17319", "17320", "17321", "17322", "17323", "17324", "17327", "17328",
    "17329", "17330", "17333", "17334", "17347", "17348", "17377", "17378",
    "17391", "17392", "17405", "17406", "17407", "17408", "17409", "17410",
    "17411", "17412", "17415", "17416", "17417", "17418", "17419", "17420",
    "17421", "17422", "17423", "17424", "17435", "17436", "17479", "17480",
    "17481", "17482", "17487", "17488", "17603", "17604", "17605", "17606",
    "17609", "17610", "17611", "17612", "17614", "17613", "17618", "17617",
    "17619", "17620", "17621", "17622", "17625", "17626", "17629", "17630",
    "17639", "17640", "17641", "17642", "17643", "17644", "17645", "17646",
    "17647", "17648", "17649", "17650", "17651", "17652", "17653", "17654",
    "17655", "17656", "17659", "17660", "17661", "17662", "17663", "17664",
    "17665", "17666", "17667", "17668", "17683", "17684", "17687", "17688",
    "17693", "17694", "18003", "18004", "18005", "18006", "18007", "18008",
    "18009", "18010", "18011", "18012", "18013", "18014", "18019", "18020",
    "18021", "18022", "18023", "18024", "18027", "18028", "18029", "18030",
    "18033", "18034", "18035", "18036", "18037", "18038", "18043", "18044",
    "18045", "18046", "18047", "18048", "18049", "18050", "18051", "18052",
    "18085", "18086", "18101", "18102", "18103", "18104", "18105", "18106",
    "18107", "18108", "18109", "18110", "18111", "18112", "18113", "18114",
    "18115", "18116", "18117", "18118", "18119", "18120", "18125", "18126",
    "18175", "18176", "18181", "18182", "18183", "18184", "18185", "18186",
    "18189", "18190", "18201", "18202", "18203", "18204", "18205", "18206",
    "18207", "18208", "18213", "18214", "18233", "18234", "18235", "18236",
    "18237", "18238", "18239", "18240", "18241", "18242", "18247", "18248",
    "18249", "18250", "18251", "18252", "18257", "18258", "18301", "18302",
    "18303", "18304", "18309", "18310", "18311", "18312", "18313", "18314",
    "18409", "18410", "18413", "18414", "18415", "18416", "18417", "18418",
    "18419", "18420", "18421", "18422", "18423", "18424", "18425", "18426",
    "18427", "18428", "18447", "18448", "18449", "18450", "18451", "18452",
    "18463", "18464", "18477", "18478", "18503", "18504", "18511", "18512",
    "18513", "18514", "18517", "18518", "18519", "18520", "18525", "18526",
    "18529", "18530", "18567", "18568", "18573", "18574", "18601", "18602",
    "18603", "18604", "18609", "18610", "18611", "18612", "18613", "18614",
    "18615", "18616", "18619", "18620", "18621", "18622", "18623", "18624",
    "18625", "18626", "18627", "18628", "18629", "18630", "18631", "18632",
    "18633", "18634", "18635", "18636", "18637", "18638", "18639", "18640",
    "18755", "18756", "19001", "19002", "19003", "19004", "19005", "19006",
    "19007", "19008", "19011", "19012", "19013", "19014", "19015", "19016",
    "19019", "19020", "19021", "19022", "19027", "19028", "19031", "19032",
    "19033", "19034", "19035", "19036", "19037", "19038", "19045", "19046",
    "19051", "19052", "19053", "19054", "19091", "19092", "19101", "19102",
    "19103", "19104", "19105", "19106", "19107", "19108", "19109", "19110",
    "19119", "19120", "19165", "19166", "19167", "19168", "19201", "19202",
    "19203", "19204", "19205", "19206", "19207", "19208", "19209", "19210",
    "19217", "19218", "19223", "19224", "19251", "19252", "19255", "19256",
    "19259", "19260", "19269", "19270", "19271", "19272", "19301", "19302",
    "19303", "19304", "19305", "19306", "19307", "19308", "19309", "19310",
    "19313", "19314", "19315", "19316", "19319", "19320", "19321", "19322",
    "19323", "19324", "19325", "19326", "19327", "19328", "19333", "19334",
    "19337", "19338", "19339", "19340", "19341", "19342", "19343", "19344",
    "19345", "19346", "19401", "19402", "19405", "19406", "19409", "19410",
    "19411", "19412", "19413", "19414", "19415", "19416", "19417", "19418",
    "19421", "19422", "19425", "19426", "19435", "19436", "19483", "19484",
    "19489", "19490", "19565", "19566", "19567", "19568", "19571", "19572",
    "19575", "19576", "19577", "19578", "19601", "19602", "19603", "19604",
    "19605", "19606", "19607", "19608", "19609", "19610", "19611", "19612",
    "19613", "19614", "19615", "19616", "19617", "19618", "19619", "19620",
    "19621", "19622", "19623", "19624", "19665", "19666", "19667", "19668",
    "19669", "19670", "19701", "19702", "19711", "19712", "19713", "19714",
    "19715", "19716", "19719", "19720", "19721", "19722", "19735", "19736",
    "19801", "19802", "19803", "19804", "19807", "19808", "19811", "19812",
    "19813", "19814", "19815", "19816", "19817", "19818", "19819", "19820",
    "19821", "19822", "20101", "20102", "20103", "20104", "20111", "20112",
    "20151", "20152", "20155", "20156", "20171", "20172", "20173", "20174",
    "20176", "20175", "20401", "20402", "20403", "20404", "20407", "20408",
    "20409", "20410", "20411", "20412", "20413", "20414", "20415", "20416",
    "20423", "20424", "20433", "20434", "20451", "20452", "20471", "20472",
    "20473", "20474", "20475", "20476", "20481", "20482", "20483", "20484",
    "20485", "20486", "20487", "20488", "20489", "20490", "20491", "20492",
    "20493", "20494", "20495", "20496", "20497", "20498", "20501", "20502",
    "20503", "20504", "20505", "20506", "20507", "20508", "20601", "20602",
    "20605", "20606", "20607", "20608", "20623", "20624", "20625", "20626",
    "20627", "20628", "20629", "20630", "20631", "20632", "20633", "20634",
    "20635", "20636", "20641", "20642", "20643", "20644", "20645", "20646",
    "20651", "20652", "20653", "20654", "20655", "20656", "20657", "20658",
    "20659", "20660", "20661", "20662", "20663", "20664", "20665", "20666",
    "20667", "20668", "20669", "20670", "20671", "20672", "20673", "20674",
    "20675", "20676", "20677", "20678", "20679", "20680", "20681", "20682",
    "20683", "20684", "20685", "20686", "20687", "20688", "20689", "20690",
    "20691", "20692", "20701", "20702", "20703", "20704", "20705", "20706",
    "20707", "20708", "20801", "20802", "20803", "20804", "20805", "20806",
    "20807", "20808", "20809", "20810", "20811", "20812", "20813", "20814",
    "20815", "20816", "20817", "20818", "20819", "20820", "20821", "20822",
    "20823", "20824", "20825", "20826", "20827", "20828", "20829", "20830",
    "20831", "20832", "20833", "20834", "20835", "20836", "20837", "20838",
    "20839", "20840", "20841", "20842", "20843", "20844", "20845", "20846",
    "20847", "20848", "20849", "20850", "20851", "20852", "20853", "20854",
    "20857", "20858", "20861", "20862", "20871", "20872", "20887", "20888",
    "20889", "20890", "20891", "20892", "20893", "20894", "20895", "20896",
    "20897", "20898", "20901", "20902", "20903", "20904", "20905", "20906",
    "20907", "20908", "20909", "20910", "20911", "20912", "20913", "20914",
    "20915", "20916", "20917", "20918", "20919", "20920", "20921", "20922",
    "20923", "20924", "20925", "20926", "20927", "20928", "20929", "20930",
    "20931", "20932", "20933", "20934", "20935", "20936", "20937", "20938",
    "20939", "20940", "20941", "20942", "20943", "20944", "20945", "20946",
    "20947", "20948", "20949", "20950", "20951", "20952", "20953", "20954",
    "20957", "20958", "20959", "20960", "20961", "20962", "20963", "20964",
    "20965", "20966", "20967", "20968", "20971", "20972", "20975", "20976",
    "20977", "20978", "20979", "20980", "20981", "20982", "20983", "20984",
    "20985", "20986", "20987", "20988", "20989", "20990", "21125", "21126",
    "21893", "21894", "21895", "21896", "21901", "21902", "21903", "21904",
    "22101", "22102", "22103", "22104", "22105", "22106", "22107", "22108",
    "22109", "22110", "22113", "22114", "22115", "22116", "22117", "22118",
    "22119", "22120", "22121", "22122", "22123", "22124", "22125", "22126",
    "22129", "22130", "22131", "22132", "22135", "22136", "22137", "22138",
    "22139", "22140", "22141", "22142", "22143", "22144", "22145", "22146",
    "22147", "22148", "22149", "22150", "22151", "22152", "22153", "22154",
    "22155", "22156", "22157", "22158", "22159", "22160", "22161", "22162",
    "22163", "22164", "22165", "22166", "22167", "22168", "22169", "22170",
    "22171", "22172", "22173", "22174", "22175", "22176", "22177", "22178",
    "22179", "22180", "22181", "22182", "22183", "22184", "22185", "22186",
    "22187", "22188", "22189", "22190", "22191", "22192", "22193", "22194",
    "22195", "22196", "22197", "22198", "22199", "22200", "22201", "22202",
    "22203", "22204", "22207", "22208", "22209", "22210", "22213", "22214",
    "22221", "22222", "22223", "22224", "22225", "22226", "22227", "22228",
    "22229", "22230", "22231", "22232", "22233", "22234", "22301", "22302",
    "22303", "22304", "22305", "22306", "22307", "22308", "22309", "22310",
    "22311", "22312", "22317", "22318", "22321", "22322", "22323", "22324",
    "22325", "22326", "22329", "22330", "22345", "22346", "22347", "22348",
    "22349", "22350", "22351", "22352", "22353", "22354", "22355", "22356",
    "22357", "22358", "22359", "22360", "22361", "22362", "22387", "22388",
    "22401", "22402", "22403", "22404", "22405", "22406", "22407", "22408",
    "22409", "22410", "22411", "22412", "22413", "22414", "22415", "22416",
    "22417", "22418", "22419", "22420", "22421", "22422", "22423", "22424",
    "22425", "22426", "22427", "22428", "22429", "22430", "22431", "22432",
    "22433", "22434", "22435", "22436", "22437", "22438", "22439", "22440",
    "22441", "22442", "22443", "22444", "22445", "22446", "22447", "22448",
    "22449", "22450", "22451", "22452", "22453", "22454", "22455", "22456",
    "22457", "22458", "22459", "22460", "22461", "22462", "22463", "22464",
    "22465", "22466", "22467", "22468", "22469", "22470", "22471", "22472",
    "22473", "22474", "22475", "22476", "22477", "22478", "22479", "22480",
    "22481", "22482", "22483", "22484", "22485", "22486", "22487", "22488",
    "22489", "22490", "22497", "22498", "22499", "22500", "22501", "22502",
    "22503", "22504", "22511", "22512", "22533", "22534", "22535", "22536",
    "22537", "22538", "22539", "22540", "22541", "22542", "22543", "22544",
    "22545", "22546", "22547", "22548", "22549", "22550", "22551", "22552",
    "22553", "22554", "22563", "22564", "22581", "22582", "22583", "22584",
    "22601", "22602", "22603", "22604", "22605", "22606", "22609", "22610",
    "22611", "22612", "22613", "22614", "22615", "22616", "22617", "22618",
    "22619", "22620", "22621", "22622", "22623", "22624", "22625", "22626",
    "22627", "22628", "22629", "22630", "22631", "22632", "22633", "22634",
    "22637", "22638", "22639", "22640", "22641", "22642", "22643", "22644",
    "22645", "22646", "22647", "22648", "22649", "22650", "22651", "22652",
    "22653", "22654", "22655", "22656", "22657", "22658", "22659", "22660",
    "22661", "22662", "22663", "22664", "22665", "22666", "22667", "22668",
    "22669", "22670", "22671", "22672", "22673", "22674", "22675", "22676",
    "22679", "22680", "22681", "22682", "22683", "22684", "22685", "22686",
    "22687", "22688", "22689", "22690", "22691", "22692", "22697", "22698",
    "22701", "22702", "22705", "22706", "22707", "22708", "22709", "22710",
    "22715", "22716", "22717", "22718", "22723", "22724", "22731", "22732",
    "22737", "22738", "22801", "22802", "22803", "22804", "22805", "22806",
    "22807", "22808", "22809", "22810", "22811", "22812", "22815", "22816",
    "22817", "22818", "22819", "22820", "22823", "22824", "22825", "22826",
    "22827", "22828", "22829", "22830", "22831", "22832", "22833", "22834",
    "22835", "22836", "22837", "22838", "22839", "22840", "22841", "22842",
    "22843", "22844", "22845", "22846", "22847", "22848", "22849", "22850",
    "22851", "22852", "22853", "22854", "22855", "22856", "22857", "22858",
    "22859", "22860", "22861", "22862", "22863", "22864", "22865", "22866",
    "22867", "22868", "22869", "22870", "22871", "22872", "22873", "22874",
    "22877", "22878", "22879", "22880", "22881", "22882", "22883", "22884",
    "22887", "22888", "22889", "22890", "22891", "22892", "22893", "22894",
    "22895", "22896", "22897", "22898", "22901", "22902", "22903", "22904",
    "22905", "22906", "22907", "22908", "22909", "22910", "22911", "22912",
    "22913", "22914", "22915", "22916", "22917", "22918", "22919", "22920",
    "22921", "22922", "22923", "22924", "22925", "22926", "22927", "22928",
    "22929", "22930", "22931", "22932", "22933", "22934", "22935", "22936",
    "22937", "22938", "22939", "22940", "22941", "22942", "22943", "22944",
    "22945", "22946", "22947", "22948", "22949", "22950", "22951", "22952",
    "22953", "22954", "22955", "22956", "22957", "22958", "22959", "22960",
    "22961", "22962", "22963", "22964", "22965", "22966", "22967", "22968",
    "22969", "22970", "22971", "22972", "22973", "22974", "22975", "22976",
    "22977", "22978", "22981", "22982", "22983", "22984", "22985", "22986",
    "22987", "22988", "22989", "22990", "22991", "22992", "22993", "22994",
    "22995", "22996", "22997", "22998", "24203", "24204", "25013", "25014",
    "25035", "25036", "26401", "26402", "26403", "26404", "26405", "26406",
    "26461", "26462", "26101", "26102", "26301", "26302", "26471", "26472",
    "26481", "26482", "26501", "26502", "26503", "26504", "26505", "26506",
    "26651", "26652", "26751", "26752", "26901", "26902", "28181", "28182",
    "52451", "52452", "52453", "52454", "52455", "52456", "52457", "52458",
]
    

for train_no in train_no_list:
    data = scrape_train(train_no)
    if data:
        print(train_no)
        

11463
11464
11465
11466
11601
11602
11603
11604
11605
11606
11651
11652
11701
11702
11704
11705
11751
11752
11753
11754
11755
11756
11801
11802
11807
11808
11841
11842
11901
11902
11903
11904
11905
11906
12001
12002
12003
12005
12006
12007
12008
12009
12010
12011
12012
12013
12014
12015
12016
12017
12018
12019
12020
12021
12022
12023
12024
12025
12026
12027
12028
12029
12030
12031
12032
12033
12034
12035
12036
12037
12038
12039
12040
12041
12042
12045
12046
12047
12048
12049
12050
12051
12052
12053
12054
12055
12056
12057
12058
12059
12060
12061
12062
12065
12066
12067
12068
12069
12070
12071
12072
12073
12074
12075
12076
12077
12078
12079
12080
12081
12082
12083
12084
12087
12088
12089
12090
12091
12092
12097
12098
12101
12102
12103
12104
12105
12106
12107
12108
12109
12110
12111
12112
12113
12114
12115
12116
12119
12120
12121
12122
12123
12124
12125
12126
12127
12128
12129
12130
12131
12132
12133
12134
12135
12136
12137
12138
12139
12140
12141
12142
12143
12144
12145
12146
12147
1214